In [2]:
# apt install xvfb

In [3]:
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/x86_64-linux-gnu/libGL.so
!xvfb-run -s "-screen 0 1400x900x24" /bin/bash

xvfb-run: error: Xvfb failed to start


In [4]:
import os
os.environ['LD_PRELOAD'] = "/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/x86_64-linux-gnu/libGL.so"
os.environ['DISPLAY'] = ':0'

In [5]:
!echo $DISPLAY

:0


/research/agent-playground/mujoco_env_customize/gym_robotics/envs/fetch/reach.py


/research/agent-playground/mujoco_env_customize/gym_robotics/envs/robot_env.py:309: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


In [8]:
# Reach environment

# Reach environment
Dict(
    
    'achieved_goal': Box(-inf, inf, (3,), float64), 
    
    'desired_goal': Box(-inf, inf, (3,), float64), 
    
    'observation': Box(-inf, inf, (10,), float64)) : 
    [ grip_pos, object_pos.ravel(), object_rel_pos.ravel(), gripper_state, object_rot.ravel(), object_velp.ravel(), object_velr.ravel(), grip_velp, gripper_vel,]
    [3, 0, 0, 2, 0, 0, 0, 3, 2]

    so, [grip_pos 3, gripper_state 2, grip_velp 3, gripper_vel 2]



In [9]:
from jesnk_utils.rgb_to_video import RGB2VIDEO
# reset environment
env.reset()
env.render_mode = 'rgb_array'
img = env.render()

rgb_to_video = RGB2VIDEO()

# get frames for video, random actions, 30 frames
frames = []
for i in range(30):
    action = env.action_space.sample()
    obs, reward, terminated, info = env.step(action)
    # obs, reward, terminated, truncated, info
    # obs : ['observation', 'achieved_goal', 'desired_goal']
    img = env.render()
    frames.append(img)
    
# save video
rgb_to_video.set_frames(frames)
rgb_to_video.save(mode='gif')

In [ ]:
print("Hello")

In [1]:
# train with SAC, stable baseline3
import stable_baselines3
from stable_baselines3 import SAC, PPO
from stable_baselines3.sac import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
import wandb
# pip install gym-robotics
import matplotlib.pyplot as plt
from gym_robotics.envs.fetch.reach import MujocoPyFetchReachEnv
from gym.wrappers import TimeLimit


# init mujoco fetch environment
env = MujocoPyFetchReachEnv()

log_dir = "./sac_fetch_reach_tensorboard/"


max_steps = 30_000

config = {
    "policy_type": "PPO",
    "total_timesteps": max_steps,
    "env_name": "FetchReach",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


# init mujoco fetch environment
# init mujoco fetch environment
env = MujocoPyFetchReachEnv(reward_type='dense')
env = Monitor(env, log_dir)
#env = TimeLimit(env, max_episode_steps=100)
env = DummyVecEnv([lambda: env])

env_eval = MujocoPyFetchReachEnv(reward_type='dense')
env_eval = Monitor(env_eval, log_dir)
#env_eval = TimeLimit(env_eval, max_episode_steps=100)
env_eval = DummyVecEnv([lambda: env_eval])

env.render_mode = 'rgb_array'
# wrap environment
# init model
model = SAC(MlpPolicy, env, verbose=1, 
            device='cuda',wandb_log=True)
#model = PPO(MlpPolicy, env, verbose=1,
#            device='cuda')

# train model
model.learn(total_timesteps=max_steps, 
            log_interval=10, 
            tb_log_name="sac_fetch_reach", 
            reset_num_timesteps=False, 
            eval_freq=100, 
            n_eval_episodes=10,
            eval_log_path="sac_fetch_reach_eval",
            eval_env=env_eval,
            )

model.save(f"sac_fetch_reach_{max_steps}")

/research/agent-playground/mujoco_env_customize/gym_robotics/envs/robot_env.py:313: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/research/agent-playground/mujoco_env_customize/gym_robotics/envs/fetch/reach.py


wandb: Currently logged in as: jesnk. Use `wandb login --relogin` to force relogin


/research/agent-playground/mujoco_env_customize/gym_robotics/envs/robot_env.py:313: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(


Using cuda device


/env/lib/python3.8/site-packages/numpy/linalg/linalg.py:2517: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (ord in ('f', 'fro') and ndim == 2) or


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# plot training
from stable_baselines3.common.results_plotter import load_results, ts2xy
import matplotlib.pyplot as plt

# load results
results = load_results("./sac_fetch_reach_tensorboard")

# plot results
plt.plot(results)
plt.show()

In [10]:
# save model
#model.save("sac_fetch_reach")
# load model
# train with SAC, stable baseline3
import stable_baselines3
from stable_baselines3 import SAC, PPO
from stable_baselines3.sac import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from wandb.integration.sb3 import WandbCallback
import wandb
# pip install gym-robotics
import matplotlib.pyplot as plt
from gym_robotics.envs.fetch.reach import MujocoPyFetchReachEnv
from gym.wrappers import TimeLimit
from jesnk_utils.rgb_to_video import RGB2VIDEO

model = SAC.load("sac_fetch_reach_30000")

In [1]:
# test model
rgb_to_video = RGB2VIDEO()

env = MujocoPyFetchReachEnv(reward_type='dense')
env.render_mode = 'rgb_array'
#env = Monitor(env_eval, log_dir)
env = TimeLimit(env, max_episode_steps=100)
env = DummyVecEnv([lambda: env])
env.render_mode = 'rgb_array'

obs = env.reset()
frames = []

    


episode_step = 0
episode_num = 0
for i in range(1,1001):
    
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    #print(dones)
    frame = env.render()
    
    # insert infos
    
    
    frames.append(frame)
    
    # insert infos
    
    
    if dones[0]:
        obs = env.reset()
        episode_step = 0
        episode_num += 1    

        
rgb_to_video.set_frames(frames)
rgb_to_video.save(path=f'{i}_test.gif',mode='gif')
print(f'episode {i} done')
frames = []
rgb_to_video.container.clear()

# save model

NameError: name 'RGB2VIDEO' is not defined